### Regras


In [116]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [117]:
def analise_estoque(filial):
    df_saldo_estoque = pd.read_excel(f'planilhas/estoque/saldo_estoque_{filial}.xlsx', header=11, usecols=('B,C,F,G,H,I,J,O,P,Q,S,U,X'))[0:-3]
    df_saldo_estoque = df_saldo_estoque.set_axis(['ean','produto','laboratorio','grupo','curva','estoque_minimo','demanda','estoque','preco_custo','preco_venda','lucro','preco_custo_total','preco_venda_total'], axis=1)
    df_saldo_estoque['grupo'] = df_saldo_estoque['grupo'].astype(int)
    df_saldo_estoque['preco_custo_total'] = df_saldo_estoque['preco_custo_total'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['preco_custo'] = df_saldo_estoque['preco_custo'].str.replace('.', '').str.replace(',', '.')
    df_saldo_estoque['estoque_minimo'] = pd.to_numeric(df_saldo_estoque['estoque_minimo'], errors='coerce')
    df_saldo_estoque['preco_custo_total'] = pd.to_numeric(df_saldo_estoque['preco_custo_total'], errors='coerce')
    df_saldo_estoque['preco_custo'] = pd.to_numeric(df_saldo_estoque['preco_custo'], errors='coerce')
    #df_saldo_estoque['ean'] = df_saldo_estoque['ean'].fillna(0).astype(int)

    df_saldo_estoque_filtrado = df_saldo_estoque.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_filtrado.query('estoque == 0')
    df_faltas['valor_faltas'] = df_faltas['estoque_minimo']*df_faltas['preco_custo']
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    return valor_em_estoque, valor_faltas, df_saldo_estoque

def analise_estoque_grupo(df_saldo_estoque_grupo, grupo):
    df_saldo_estoque_grupo = df_saldo_estoque_grupo.query(f'grupo == {grupo}')
    df_saldo_estoque_grupo_filtrado = df_saldo_estoque_grupo.query('estoque_minimo > 0')
    valor_em_estoque = df_saldo_estoque_grupo['preco_custo_total'].sum()
    df_faltas = df_saldo_estoque_grupo_filtrado
    df_faltas['valor_faltas'] = df_faltas['estoque_minimo']*df_faltas['preco_custo']
    df_faltas['ean'] = df_faltas['ean'].astype(str)
    valor_faltas = df_faltas['valor_faltas'].sum()
    
    df_excesso = df_saldo_estoque_grupo.query('estoque > 0')
    df_excesso['excesso'] = np.where(df_excesso['estoque']>df_excesso['demanda'],df_excesso['estoque']-df_excesso['demanda'],0)
    df_excesso = df_excesso.query('excesso > 0')
    df_excesso['ean'] = df_excesso['ean'].astype(str)
    return valor_em_estoque, valor_faltas, df_faltas, df_excesso

##### somar demandas de Centrinho, Matriz e Canasvieiras

In [118]:
valor_em_estoque_matriz, valor_faltas_matriz, df_estoque_matriz = analise_estoque('001')

In [119]:
valor_em_estoque_centrinho, valor_faltas_centrinho, df_estoque_centrinho = analise_estoque('004')

In [120]:
valor_em_estoque_canasvieiras, valor_faltas_canasvieiras, df_estoque_canasvieiras = analise_estoque('009')

In [121]:
#filtrando perfumaria
df_estoque_matriz = df_estoque_matriz.query('grupo == 10000')
df_estoque_centrinho = df_estoque_centrinho.query('grupo == 10000')
df_estoque_canasvieiras = df_estoque_canasvieiras.query('grupo == 10000')

In [122]:
df_estoque_matriz = df_estoque_matriz[['ean','produto','estoque_minimo','demanda']]
df_estoque_matriz

,ean,produto,estoque_minimo,demanda
32,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0
52,7.622301e+12,5STAR CHOC CAR 40G,0,0.0
53,7.790580e+12,7BELO BALA GELAT FRAM 70G,0,0.0
131,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0
390,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0
...,...,...,...,...
22016,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0
22017,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0
22018,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0
22019,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0


In [123]:
df_estoque_centrinho = df_estoque_centrinho[['ean','produto','estoque_minimo','demanda']]
df_estoque_centrinho

,ean,produto,estoque_minimo,demanda
32,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0
52,7.622301e+12,5STAR CHOC CAR 40G,0,0.0
53,7.790580e+12,7BELO BALA GELAT FRAM 70G,1,1.0
131,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0
390,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0
...,...,...,...,...
22016,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0
22017,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0
22018,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0
22019,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0


In [124]:
df_estoque_canasvieiras = df_estoque_canasvieiras[['ean','produto','estoque_minimo','demanda']]
df_estoque_canasvieiras

,ean,produto,estoque_minimo,demanda
32,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0
52,7.622301e+12,5STAR CHOC CAR 40G,0,0.0
53,7.790580e+12,7BELO BALA GELAT FRAM 70G,0,0.0
131,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0
390,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0
...,...,...,...,...
22016,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0
22017,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0
22018,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0
22019,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0


In [125]:
df_geral = pd.merge(df_estoque_matriz,df_estoque_centrinho, on=['ean','produto'], how='outer')
df_geral

,ean,produto,estoque_minimo_x,demanda_x,estoque_minimo_y,demanda_y
0,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0,0,0.0
1,7.622301e+12,5STAR CHOC CAR 40G,0,0.0,0,0.0
2,7.790580e+12,7BELO BALA GELAT FRAM 70G,0,0.0,1,1.0
3,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0,0,0.0
4,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0,0,0.0
...,...,...,...,...,...,...
1045,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0,0,0.0
1046,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0,0,0.0
1047,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0,0,0.0
1048,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0,0,0.0


In [126]:
df_geral['estoque_minimo'] = df_geral['estoque_minimo_x'] + df_geral['estoque_minimo_y']
df_geral['demanda'] = df_geral['demanda_x'] + df_geral['demanda_y']
df_geral = df_geral[['ean','produto','estoque_minimo','demanda']]
df_geral

,ean,produto,estoque_minimo,demanda
0,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0
1,7.622301e+12,5STAR CHOC CAR 40G,0,0.0
2,7.790580e+12,7BELO BALA GELAT FRAM 70G,1,1.0
3,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0
4,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0
...,...,...,...,...
1045,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0
1046,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0
1047,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0
1048,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0


In [127]:
df_estoque_canasvieiras

,ean,produto,estoque_minimo,demanda
32,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0
52,7.622301e+12,5STAR CHOC CAR 40G,0,0.0
53,7.790580e+12,7BELO BALA GELAT FRAM 70G,0,0.0
131,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0
390,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0
...,...,...,...,...
22016,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0
22017,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0
22018,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0
22019,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0


In [128]:
df_geral = pd.merge(df_geral,df_estoque_canasvieiras, on=['ean','produto'], how='outer')

In [129]:
df_geral['estoque_minimo'] = df_geral['estoque_minimo_x'] + df_geral['estoque_minimo_y']
df_geral['demanda'] = df_geral['demanda_x'] + df_geral['demanda_y']
df_geral = df_geral[['ean','produto','estoque_minimo','demanda']]
df_geral

,ean,produto,estoque_minimo,demanda
0,7.896008e+12,*FR HUGGIES L.SWI MA/PIS P 1UN,0,0.0
1,7.622301e+12,5STAR CHOC CAR 40G,0,0.0
2,7.790580e+12,7BELO BALA GELAT FRAM 70G,1,1.0
3,3.789610e+13,ABS MILI PR TOT SUA AB 32UN,0,0.0
4,7.896095e+12,ADOC ZERO-CAL STEVIA LIQ 80ML,0,0.0
...,...,...,...,...
1045,7.895503e+12,WHEY ISOLATE 900G BUNILHA - XC,0,0.0
1046,7.898637e+12,WHEY PRO CHOC SU PO REF 900G,0,0.0
1047,7.898637e+12,WHEY PRO CR BAU PO REF 900G,0,0.0
1048,7.899732e+12,WHEY PROT 3W CHOC AM 900G,0,0.0


In [130]:
df_geral = df_geral.query('estoque_minimo > 0')

In [131]:
filename = 'planilhas/estoque/demanda_trindade_10000.xlsx'
df_geral.to_excel(filename)